# Project 1:Basic Recommender System

## Introduction

For this project, a basic Recommender System was attempted. The data used for this was the Movielens 100k Dataset provided by Grouplens. The data set is quite diverse(a bit on the old side), but provides 100k ratings from 1700 movies (pre-1995). 

## Code:

### Opening and Cleaning Data

In [13]:
import pandas as pd 
import numpy as np
import math
import csv


In [14]:
#Data file containing ratings data
f = open("data/u.data")
text = csv.reader(f, delimiter = "\t")
ratings = list(text)

# My Ratings of the movie Data (I pick)
my_ratings = [4,5,5,4,0,2,2,1,4,4,5,3,2,2,3,0,2,5,2,1]

ratings_pd = pd.DataFrame(ratings, columns = ["user_id","movie_id", "rating","timestamp"])
ratings_pd = ratings_pd.drop(["timestamp"], 1)

movies = ratings_pd.pivot(index = "movie_id", columns = "user_id", values = "rating")

i = ['55','56','64','71','94','102','118','133','141','168','173','196','1278','928','780',
     '651','514','483','432','404']
movies = movies.loc[i,:]

l = ['13','128','201','207','222','23','234','246','267','268','269','276','493','642',
     '655','846','95','262','194','130','1']
movies = movies.loc[:,l]
movies = movies.fillna(value=0)

movies_np = np.array(movies)
movies_np = movies_np.astype(float)
movies_np = np.insert(movies_np, 0, my_ratings, axis=1)

### Processing Data: Item - Item Collobrative Filter

In [15]:
mean_movie= []
count_movie = []

for row in movies_np:
    
    row_sum = np.sum(row)
    count = np.count_nonzero(row)
    count_movie.append(count)
    mean_movie.append(row_sum/count)

   
count_user = []
mean_user = []

for row in movies_np.T:
    row_sum = np.sum(row)
    count = np.count_nonzero(row)
    count_user.append(count)
    mean_user.append(row_sum/count)

    
movies_np[movies_np==0] = np.nan


mean_cent = []
i = 0
for row in  movies_np:
    mean_cent.append(row - mean_movie[i]) 
    i += 1
    
mean_cent = np.array(mean_cent)

mean_cent = np.nan_to_num(mean_cent)
U = []
 
for row in mean_cent:
    y = 0
    for val in row:
        x = math.pow(val,2)
        y = y + x
    U.append(math.sqrt(y))

cent_t = mean_cent.T

num = np.dot(mean_cent,mean_cent.T)

for i in range(20):
    for j in range(20):
        num[i,j] = num[i,j]/(U[i]*U[j])
        

In [16]:
pd.DataFrame(num)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.306038,0.078072,-0.006584,0.172497,-0.178380,0.208471,-0.048302,0.011269,0.031623,0.050412,0.286860,0.452375,0.117112,0.099085,-0.152027,-0.427832,0.110602,0.005406,0.130236
1,0.306038,1.000000,0.035153,0.112816,0.312059,-0.088840,0.255727,-0.282705,0.112033,0.098613,-0.252062,0.100122,0.100156,0.016741,-0.016864,-0.170010,-0.169492,-0.089424,0.209517,0.142462
2,0.078072,0.035153,1.000000,0.414943,-0.036370,0.187642,0.513200,-0.062090,0.032075,0.090010,0.445650,0.334719,0.152542,0.293972,-0.052881,0.273594,-0.099313,0.381693,-0.102576,-0.065566
3,-0.006584,0.112816,0.414943,1.000000,0.452156,0.515388,0.329212,0.024683,0.389490,0.098519,0.079781,0.215543,0.319099,0.550183,0.332283,0.152554,-0.121007,-0.008961,-0.255050,0.115198
4,0.172497,0.312059,-0.036370,0.452156,1.000000,0.021678,0.251976,-0.121942,0.188982,-0.429315,-0.205939,0.159364,0.246718,0.113408,-0.083085,0.109659,-0.278639,-0.281912,-0.060437,0.435836
5,-0.178380,-0.088840,0.187642,0.515388,0.021678,1.000000,0.000000,0.462600,0.267652,0.191606,0.140625,-0.090685,0.128359,0.553818,0.275268,0.276225,-0.039463,-0.367507,-0.213988,0.141290
6,0.208471,0.255727,0.513200,0.329212,0.251976,0.000000,1.000000,0.180518,-0.222222,-0.111359,-0.047795,0.421637,0.466252,0.127294,-0.116018,0.328813,0.172016,0.423790,0.017767,0.340693
7,-0.048302,-0.282705,-0.062090,0.024683,-0.121942,0.462600,0.180518,1.000000,-0.038408,0.286394,-0.017348,0.099595,0.206299,0.375904,0.164638,0.095610,0.353370,0.030471,-0.060187,0.258487
8,0.011269,0.112033,0.032075,0.389490,0.188982,0.267652,-0.222222,-0.038408,1.000000,0.267261,0.456442,-0.070273,-0.124334,0.281865,0.616726,-0.309471,-0.589768,-0.214720,-0.284268,-0.366900
9,0.031623,0.098613,0.090010,0.098519,-0.429315,0.191606,-0.111359,0.286394,0.267261,1.000000,0.417701,0.035215,-0.037383,0.204124,0.188490,-0.383825,-0.055824,-0.054366,0.028490,-0.220631


### Referrences: 

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. http://dx.doi.org/10.1145/2827872 http://grouplens.org/datasets/movielens/